# Assignment
- Start a clean notebook, or continue with last module's assignment notebook.
- Continue to participate in our Kaggle competition with the Tanzania Waterpumps data. 
- Do more exploratory data analysis, data cleaning, feature engineering, and feature selection.
- Try a Decision Tree Classifier. 
- Submit new predictions.
- Commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- Create visualizations and share on Slack.
- Read more about decision trees and tree ensembles. You can start with the links at the top of this notebook.
- Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.


In [0]:
!pip install category_encoders

In [0]:
import numpy as np
import pandas as pd
from math import sqrt
import pandas_profiling
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

In [92]:
LOCAL = '../data/tanzania/'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train = pd.merge(pd.read_csv(WEB + 'train_features.csv'), 
                 pd.read_csv(WEB + 'train_labels.csv'))
test = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [0]:
train.isnull().sum()

In [0]:
train['scheme_name'].value_counts()

In [107]:
train['scheme_management'].value_counts()

VWC                 29470
WUG                  4164
MISSING              3128
Water authority      2507
WUA                  2334
Water Board          2197
Parastatal           1320
Private operator      836
Company               824
Other                 603
SWC                    78
Trust                  58
None                    1
Name: scheme_management, dtype: int64

In [0]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace them with the column mean.
    cols_with_zeros = ['construction_year', 'longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col] = X[col].fillna(X[col].mean())
        
    # Convert date_recorded to datetime
    X['datetime_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract year from date_recorded
    X['year_recorded'] = X['datetime_recorded'].dt.year
    
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns=['id','quantity_group','recorded_by','datetime_recorded'])
    
    # for categoricals with missing values, fill with the category 'MISSING'
    categoricals = X.select_dtypes(exclude='number').columns
    for col in categoricals:
        X[col] = X[col].fillna('MISSING')
    
    X['gov_funder'] = X['funder']=='Government Of Tanzania'
    X['dwe_funder'] = X['funder']=='Dwe'
    X['missing_funder'] = X['funder']=='MISSING'
    X['gov_installer'] = X['installer']=='Government'
    X['dwe_installer'] = X['installer']=='DWE'
    X['missing_installer'] = X['installer']=='MISSING'
    X['Igosi_ward'] = X['ward']=='Igosi'
    X['Imalinyi_ward'] = X['ward']=='Imalinyi'
#     X['Siha_ward'] = X['ward']=='Siha Kati'
    X['none_scheme_name'] = (X['scheme_name']=='None')|(X['scheme_name']=='MISSING')
    X['gov_scheme_name'] = X['scheme_name']=='Government'
    X['Madukani_subvillage'] = X['subvillage']=='Madukani'
    X['Shuleni_subvillage'] = X['subvillage']=='Shuleni'
    X['Majengo_subvillage'] = X['subvillage']=='Majengo'
    X['none_wpt'] = (X['wpt_name']=='none')|(X['wpt_name']=='MISSING')
    X['Shuleni_wpt'] = X['wpt_name']=='Shuleni'
    
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
# categorical_features = X_train.describe(exclude='number').T.query('unique <= 2000').index.drop(['water_quality']).tolist()
# numeric_features = X_train.select_dtypes('number').columns.drop(['region_code','district_code']).tolist()
# features = categorical_features + numeric_features

In [94]:
train.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,year_recorded
count,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000
mean,321.925261,669.567656,35.149033,-5.884512,0.477736,15.258291,5.616751,179.805513,1996.825469,2011.921907
std,3197.240487,693.005745,2.564423,2.762702,13.312977,17.530228,9.621230,463.081564,10.095900,0.959690
min,0.000000,-63.000000,29.607122,-11.649440,0.000000,1.000000,0.000000,0.000000,1960.000000,2002.000000
25%,0.000000,0.000000,33.353813,-8.528215,0.000000,5.000000,2.000000,0.000000,1996.000000,2011.000000
50%,0.000000,372.500000,35.149033,-5.429301,0.000000,12.000000,3.000000,25.000000,1996.825469,2012.000000
75%,25.000000,1320.000000,37.170578,-3.400154,0.000000,17.000000,5.000000,215.000000,2004.000000,2013.000000
max,350000.000000,2770.000000,40.344301,-0.998464,1776.000000,99.000000,80.000000,15300.000000,2013.000000,2013.000000


In [95]:
train.describe(exclude='number').sort_values(by='unique', axis=1)

,Shuleni_wpt,gov_funder,dwe_funder,missing_funder,gov_installer,missing_installer,Igosi_ward,dwe_installer,none_scheme_name,gov_scheme_name,Madukani_subvillage,Shuleni_subvillage,Majengo_subvillage,Imalinyi_ward,none_wpt,public_meeting,permit,status_group,source_class,quantity,management_group,waterpoint_type_group,quality_group,source_type,payment_type,waterpoint_type,payment,extraction_type_class,water_quality,basin,source,management,scheme_management,extraction_type_group,extraction_type,region,lga,date_recorded,funder,installer,ward,scheme_name,subvillage,wpt_name
count,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,5,5,6,6,7,7,7,7,7,8,9,10,12,13,13,18,21,124,349,1717,1930,2082,2564,17232,30661
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,functional,groundwater,enough,user-group,communal standpipe,good,spring,never pay,communal standpipe,never pay,gravity,soft,Lake Victoria,spring,vwc,VWC,gravity,gravity,Iringa,Njombe,2011-03-17,Government Of Tanzania,DWE,Igosi,MISSING,Shuleni,none
freq,46104,40199,47130,44616,46062,44603,47263,33542,24470,47269,47116,47100,47111,47311,44641,40838,31071,25807,36638,26567,42027,27642,40598,13620,20287,22778,20287,21448,40598,8137,13620,32449,29470,21448,21448,4250,2003,474,7321,13978,257,22532,420,2879


In [0]:
train_features.sample(1)

In [104]:
train['extraction_type_group'].value_counts()

gravity            21448
nira/tanira         6526
other               5193
submersible         4894
swn 80              2930
mono                2261
india mark ii       1928
afridev             1435
rope pump            353
other handpump       292
other motorpump       94
wind-powered          93
india mark iii        73
Name: extraction_type_group, dtype: int64

In [113]:
target = 'status_group'
train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
categorical_features = train_features.describe(exclude='number').T.query(
    'unique <= 200').drop(
    ['public_meeting','extraction_type_group','water_quality','quality_group']).index.tolist()
features = numeric_features + categorical_features

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_encoded)
# X_val_scaled = scaler.transform(X_val_encoded)

model_dt = DecisionTreeClassifier(min_samples_leaf=27, random_state=42)
model_dt.fit(X_train_encoded, y_train)
train_score = model_dt.score(X_train_encoded, y_train)
val_score = model_dt.score(X_val_encoded, y_val)
print('Train Accuracy: {:5.4f}%'.format(train_score*100))
print('Validation Accuracy: {:5.4f}%'.format(val_score*100))

Train Accuracy: 79.9053%
Validation Accuracy: 77.5000%


0. Original
- Train Accuracy: 79.9095%
- Validation Accuracy: 77.3990%
1. public_meeting
  - Train Accuracy: 79.8843%
  - Validation Accuracy: 77.4242%
2. extraction_type_group
  - Train Accuracy: 79.8843%
  - Validation Accuracy: 77.4327%
3. water_quality
  - Train Accuracy: 79.8927%
  - Validation Accuracy: 77.4579%
4. quality_group
  - Train Accuracy: 79.9053%
  - Validation Accuracy: 77.5000%

In [114]:
new_f_score = []
for feature in features:
  f_drop = features.copy()
  f_drop.remove(feature)
  X_train_subset = X_train[f_drop]
  X_val_subset = X_val[f_drop]

  encoder = ce.OneHotEncoder(use_cat_names=True)
  X_train_encoded = encoder.fit_transform(X_train_subset)
  X_val_encoded = encoder.transform(X_val_subset)

  scaler = RobustScaler()
  X_train_scaled = scaler.fit_transform(X_train_encoded)
  X_val_scaled = scaler.transform(X_val_encoded)

  model_dt_drop = DecisionTreeClassifier(min_samples_leaf=27, random_state=42)
  model_dt_drop.fit(X_train_scaled, y_train)
  print(feature, model_dt_drop.score(X_val_scaled, y_val)*100)
  if model_dt_drop.score(X_val_scaled, y_val) > val_score:
    val_score = model_dt_drop.score(X_val_scaled, y_val)
    new_f_score = [feature, val_score]
new_f_score

amount_tsh 76.27946127946129
gps_height 77.24747474747474
longitude 76.86868686868686
latitude 77.16329966329967
num_private 77.52525252525253
region_code 77.33164983164983
district_code 77.28956228956228
population 76.75925925925927
construction_year 76.69191919191918
year_recorded 77.35690235690235
basin 76.91919191919192
region 77.08754208754208
lga 76.65824915824916
scheme_management 77.5
permit 77.35690235690235
extraction_type 77.28956228956228
extraction_type_class 77.28114478114479
management 77.46632996632997
management_group 77.51683501683502
payment 77.5
payment_type 77.5
quantity 72.38215488215488
source 77.46632996632997
source_type 77.49158249158249
source_class 77.32323232323233
waterpoint_type 76.68350168350169
waterpoint_type_group 77.52525252525253
gov_funder 77.28114478114479
dwe_funder 77.52525252525253
missing_funder 77.5
gov_installer 77.39057239057239
dwe_installer 77.39057239057239
missing_installer 77.52525252525253
Igosi_ward 77.52525252525253
Imalinyi_ward 77

['num_private', 0.7752525252525253]

### RobustScaler

Decision Tree Classifier (min_samples_leaf=27)
- Train Accuracy: 79.7917%
- Validation Accuracy: 77.0202%


## Submit to predictive modeling competition


### Write submission CSV file

The format for the submission file is simply the row id and the predicted label (for an example, see `sample_submission.csv` on the data download page.

For example, if you just predicted that all the waterpoints were functional you would have the following predictions:

<pre>id,status_group
50785,functional
51630,functional
17168,functional
45559,functional
49871,functional
</pre>

Your code to generate a submission file may look like this: 
<pre># estimator is your scikit-learn estimator, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = estimator.predict(X_test)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
</pre>

In [77]:
X_test_subset = test[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)
all(X_test_encoded.columns == X_train_encoded.columns)

True

In [0]:
y_pred = model_dt.predict(X_test_scaled)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-03.csv', index=False)

In [80]:
!head submission-03.csv

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,non functional
28965,non functional
36301,non functional


In [0]:
# submission-01: 71.9% 'unique <= 3000' non-scaler

### Send submission CSV file to Kaggle

#### Option 1. Kaggle web UI
 
Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.


#### Option 2. Kaggle API

Use the Kaggle API to upload your CSV file.